In [172]:
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


* Define a variable for URL 
* Print URL value and make sure URL is valid

In [173]:
# Define a variable for URL 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
print(results)

<Response [200]>


 * Get the HTML page contents
 * Convert the page to BeautifulSoup for parsing

In [174]:
 #get the HTML page contents
ca_postal_code_page = results.text

# Covert the page to SOUP
html_soup_page = BeautifulSoup(ca_postal_code_page, 'lxml')


* Get the HTML Postal Codes table data
* Assign table rows to a separate variable

In [175]:
# Get the HTML Postal Codes table data
table_detail = html_soup_page.find('table', class_='wikitable sortable')
table_rows = table_detail.find_all('tr')

* Scan through Toronto City postal codes and filter some data

In [176]:
# Scan through Toronto City postal codes and filter some data
a='Not assigned'
res = []
tab_row=[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    
    if row:
       if row[1] != a:
          if row[2] == a:
              row[2] = row[1] 
          res.append(row)
      
df = pd.DataFrame(res, columns=['PostalCode','Borough','Neighbourhood'])
df.shape


(211, 3)

* Combine neighborhoods by postal code. All neighborhoods under one postal code will be combined together.

In [177]:
# Combine rows by postal 
df_combine = df.groupby(['PostalCode','Borough'])['Neighbourhood'].agg(lambda col: ', '.join(col.astype('str'))).reset_index()


* Get Coordinates data from CSV file and join the two dataframes on Postal Code

In [178]:
file_url = 'http://cocl.us/Geospatial_data'
df_cords = pd.read_csv(file_url, skiprows=[0], header = None)

df_cords.columns=['Code','Latitude','Longitude']

pd.merge(df_combine, df_cords, left_on="PostalCode", right_on="Code").drop('Code', axis=1)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


* Get the combined data filtered and pick only Toronto Esat (Scarborough) neighborhoods

In [179]:
# Get the combined data in a neighborhood DataFrame 
neighborhoods=pd.merge(df_combine, df_cords, left_on="PostalCode", right_on="Code").drop('Code', axis=1)

# Target the East of Toronto, especially Scarborough
east_of_to = neighborhoods[neighborhoods['Borough'].str.contains('Scarborough')].reset_index(drop=True)

In [180]:
# Foursquare connection ID
CLIENT_ID = 'TKVOOCUPD0AUTPWYELLMG1VN5TJY4K0ASQBYR3QRTYGEKACE' # your Foursquare ID
CLIENT_SECRET = 'YIMMLNYUKL5WOK2EGUOOBCA3ZPXJZDYTVCWPVMYCMJBFOZS0' # your Foursquare Secret
VERSION = '20190811' # Foursquare API version

* In the FOR LOOP we will go through Scarborough area neighborhoods to determine which neighborhood has more than 20 resturants within 1000 meters radius. 


In [181]:
# In the FOR LOOP we will go through Scarborough area neighborhoods to determine which neighborhood has more than 20 resturants within 1000 meters radius. 

east_of_to.shape
collection_center=pd.DataFrame()


for index, row in east_of_to.iterrows():

    neighborhood_latitude = east_of_to.loc[index, 'Latitude'] # neighborhood latitude value
    neighborhood_longitude = east_of_to.loc[index, 'Longitude'] # neighborhood longitude value
    
    neighborhood_name = east_of_to.loc[index, 'Neighbourhood'] # neighborhood name

      
    LIMIT = 30 # limit of number of venues returned by Foursquare API
    radius = 1000 # define radius
    resturant_cat='4d4b7105d754a06374d81259'
    #buffet='52e81612bcbc57f1066b79f4'
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={} &categoryId={} &v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        resturant_cat,
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
    #url # display URL
    
    
    results = requests.get(url).json()
    results
    
    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
   
    east_of_to.loc[index, 'Neighbourhood']    
    venues = results['response']['groups'][0]['items']
    venues
    
    nearby_venues = json_normalize(venues) # flatten JSON
  
    if not nearby_venues.apply(get_category_type, axis=1).empty:
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    
        # Make sure we have 20 or more resturant in that neighborhood
        if nearby_venues.shape[0] >= 20:
           collect_center=east_of_to.loc[index, 'Neighbourhood']
           
           
           #Copy neighborhood coordinates details into collection_center DataFrame
           collection_center = collection_center.append(east_of_to.loc[index])
           
           # Print the Neighborhood name along with number resturants found
           print(nearby_venues.shape[0], " Resturant found in ", collect_center)
       
print(collection_center)
collection_center.shape


23  Resturant found in  Dorset Park, Scarborough Town Centre, Wexford Heights
30  Resturant found in  Agincourt
22  Resturant found in  Clarks Corners, Sullivan, Tam O'Shanter
25  Resturant found in  Agincourt North, L'Amoreaux East, Milliken, Steeles East
        Borough   Latitude  Longitude  \
10  Scarborough  43.757410 -79.273304   
12  Scarborough  43.794200 -79.262029   
13  Scarborough  43.781638 -79.304302   
14  Scarborough  43.815252 -79.284577   

                                        Neighbourhood PostalCode  
10  Dorset Park, Scarborough Town Centre, Wexford ...        M1P  
12                                          Agincourt        M1S  
13            Clarks Corners, Sullivan, Tam O'Shanter        M1T  
14  Agincourt North, L'Amoreaux East, Milliken, St...        M1V  


(4, 5)

* Display the identified locations for collection centers on the map 

In [182]:
# Display the identified locations for collection centers on the map 
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, collection_center in zip(collection_center['Latitude'], collection_center['Longitude'], collection_center['Borough'], collection_center['Neighbourhood']):
    label = '{}, {}'.format(collection_center, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to